In [2]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import time
import os
from PIL import Image
from torchvision.transforms import v2
from tempfile import TemporaryDirectory
import shutil
import random

cudnn.benchmark = True
plt.ion()   # interactive mode

#### Data Transformations

In [2]:
# data transformations to loop through
minimal_transforms = {
    'synthetic_train': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=3),
        v2.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5]),
        v2.Resize((256, 256))
    ]),
    'test': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=3),
        v2.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5]),
        v2.Resize((256, 256))
    ]),
}

basic_transforms = {
    'synthetic_train': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=3),
        v2.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5]),
        v2.RandomHorizontalFlip(p=0.5),
        v2.Resize((256, 256))
    ]),
    'test': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=3),
        v2.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5]),
        v2.Resize((256, 256))
    ]),
}

auto_transforms = {
    'synthetic_train': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=3),
        v2.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5]),
        v2.AutoAugment(policy=v2.AutoAugmentPolicy.IMAGENET),
        v2.Resize((256, 256))
    ]),
    'test': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=3),
        v2.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5]),
        v2.Resize((256, 256))
    ]),
}

#### Create Training Set with Real/Synthetic Images

In [3]:
def makeSyntheticTrain(train_directory, synthetic_train_directory, train_percentage, synthetic_percentage):

    # Remove any existing images in directory
    try:
        shutil.rmtree(synthetic_train_directory)
    except:
        print("directory does not exist")

    # Loop through subfolders, generate synthetic images
    subfolders = [f for f in os.listdir(train_directory)]

    for s in subfolders:
        # for each subfolder in the train directory, make the same in the synthetic train directory
        os.makedirs(f"{synthetic_train_directory}/{s}", exist_ok=True)
        
        # get a random sample from each subfolder
        subfolder_path = f"{train_directory}/{s}"
        files = os.listdir(subfolder_path)
        sample_files = random.sample(files, round(len(files)*train_percentage))
        
        # create synthetic sample based on sampled original images
        synthetic_subfolder_path = subfolder_path.replace('train','synthetic')
        synthetic_files = [f for f in os.listdir(synthetic_subfolder_path) if int(f.replace('.png','').split('_')[1]) in [int(f.replace('.png','').split('_')[1]) for f in sample_files]]
        synthetic_sample_files = random.sample(synthetic_files, round(len(files)*synthetic_percentage))
        
        # Move sample files to synthetic directory
        for f in sample_files:
            
            image_path = f"{subfolder_path}/{f}"
            destination_directory = f"{synthetic_train_directory}/{s}/"
            shutil.copyfile(image_path, destination_directory + image_path.split('/')[-1])

        # Move synthetic sample files to synthetic directory
        for f in synthetic_sample_files:

            image_path = f"{synthetic_subfolder_path}/{f}"
            destination_directory = f"{synthetic_train_directory}/{s}/"
            shutil.copyfile(image_path, destination_directory + image_path.split('/')[-1])

#### Read Data

In [4]:
def get_data(data_dir, data_sets, data_transforms):
    
    image_datasets = {
        x: datasets.ImageFolder(
            os.path.join(data_dir, x),
            data_transforms[x]
        )
        for x in data_sets
    }

    dataloaders = {
        x: DataLoader(
            image_datasets[x], 
            batch_size=32,
            shuffle=True, 
            num_workers=4
        )
        for x in data_sets
    }

    dataset_sizes = {
        x: len(image_datasets[x]) 
        for x in data_sets
    }

    class_names = image_datasets['synthetic_train'].classes

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    return image_datasets, dataloaders, dataset_sizes, class_names, device

#### Train Model

In [5]:
def train_model(model, criterion, optimizer, dataloaders, dataset_sizes, num_epochs=10):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        
        best_test_loss = float('inf')
        patience = 2  # Number of epochs to wait for improvement before stopping
        test_losses = []
        train_losses = []
        test_acc = []
        train_acc = []
        patience_counter = 0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['synthetic_train', 'test']:
                if phase == 'synthetic_train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'synthetic_train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'synthetic_train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                
                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double().item() / dataset_sizes[phase]

                if phase =='synthetic_train':
                    train_losses.append(epoch_loss)
                    train_acc.append(epoch_acc)
                else:
                    test_losses.append(epoch_loss)
                    test_acc.append(epoch_acc)
                    
                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'test' and epoch_loss <= best_test_loss:
                    best_test_loss = epoch_loss
                    patience_counter = 0  # Reset counter
                    torch.save(model.state_dict(), best_model_params_path)
                elif phase == 'test' and epoch_loss > best_test_loss:
                    patience_counter += 1
                
                # Early stopping check
                #if patience_counter >= patience:
                #    print("Stopping early due to no improvement in validation loss.")
                #    break

        # store results in dataframe
        dat = {
            "epoch": range(len(test_losses)),
            "test_losses": test_losses,
            "train_losses": train_losses,
            "test_accuracies": test_acc,
            "train_accuracies": train_acc
        }

        result = pd.DataFrame(data=dat)
        print()
        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best test Loss: {best_test_loss:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path, weights_only=True))
    return result, model

#### Loop through different training scenarios

In [6]:
num_epochs = 10
num_sims = 10
train_percs = [0.6,0.6,0.4,0.4]
synth_percs = [0.4,0.0,0.6,0.0]
transforms = {
    'minimal': minimal_transforms,
    'basic': basic_transforms,
    'auto': auto_transforms
}

for train_percentage, synthetic_percentage in zip(train_percs, synth_percs):
    for active_transform in transforms.keys():
        df_all_results = pd.DataFrame()
        for n in range(num_sims):
    
            # make the synthetic training dataset
            makeSyntheticTrain(
                train_directory='../data/alzheimer_mri/train',
                synthetic_train_directory='../data/alzheimer_mri/synthetic_train', 
                train_percentage=train_percentage, 
                synthetic_percentage=synthetic_percentage
            )

            # get and load datasets
            data_dir = '../data/alzheimer_mri'
            data_sets = ['synthetic_train','test']
            image_datasets, dataloaders, dataset_sizes, class_names, device = get_data(
                data_dir=data_dir, 
                data_sets=data_sets, 
                data_transforms=transforms.get(active_transform)
            )

            # instantiate model
            model = models.resnet18(weights='IMAGENET1K_V1')

            # Set the size of each output sample to nn.Linear(num_ftrs, len(class_names))
            num_ftrs = model.fc.in_features
            model.fc = nn.Linear(num_ftrs, len(class_names))
            model = model.to(device)

            # train the model
            df_results,_ = model = train_model(
                model=model, 
                criterion = nn.CrossEntropyLoss(),
                optimizer = optim.Adam(model.parameters(), lr=0.001), 
                dataloaders=dataloaders,
                dataset_sizes=dataset_sizes,
                num_epochs=num_epochs
            )

            df_results['train_percentage'] = train_percentage
            df_results['synth_percentage'] = synthetic_percentage
            df_results['transform'] = active_transform
            df_results['sim_num'] = n
            df_results['category'] = df_results.apply(lambda row: row['transform']+'_'+str(row['train_percentage'])+'_'+str(row['synth_percentage']), axis=1)
            df_all_results = pd.concat([df_all_results, df_results],ignore_index=True)
        df_all_results.to_csv('../results/results_resnet_'+active_transform+'_'+str(train_percentage)+'_'+str(synthetic_percentage)+'.csv')

/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 0/9
----------
synthetic_train Loss: 1.0748 Acc: 0.4846
test Loss: 1.0632 Acc: 0.4953
Epoch 1/9
----------
synthetic_train Loss: 1.0346 Acc: 0.4896
test Loss: 1.0342 Acc: 0.5320
Epoch 2/9
----------
synthetic_train Loss: 1.0121 Acc: 0.5109
test Loss: 0.9489 Acc: 0.5375
Epoch 3/9
----------
synthetic_train Loss: 0.9993 Acc: 0.5184
test Loss: 0.9715 Acc: 0.5258
Epoch 4/9
----------
synthetic_train Loss: 1.0103 Acc: 0.5084
test Loss: 0.9371 Acc: 0.5383
Epoch 5/9
----------
synthetic_train Loss: 0.9898 Acc: 0.5281
test Loss: 0.9344 Acc: 0.5227
Epoch 6/9
----------
synthetic_train Loss: 0.9883 Acc: 0.5242
test Loss: 0.9412 Acc: 0.5711
Epoch 7/9
----------
synthetic_train Loss: 0.9791 Acc: 0.5428
test Loss: 0.9682 Acc: 0.5625
Epoch 8/9
----------
synthetic_train Loss: 0.9717 Acc: 0.5348
test Loss: 0.9140 Acc: 0.5656
Epoch 9/9
----------
synthetic_train Loss: 0.9544 Acc: 0.5486
test Loss: 0.9147 Acc: 0.5867

Training complete in 8m 14s
Best test Loss: 0.914021


/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 0/9
----------
synthetic_train Loss: 1.0662 Acc: 0.4848
test Loss: 1.0812 Acc: 0.5195
Epoch 1/9
----------
synthetic_train Loss: 1.0341 Acc: 0.4994
test Loss: 1.0691 Acc: 0.4656
Epoch 2/9
----------
synthetic_train Loss: 1.0272 Acc: 0.4893
test Loss: 0.9625 Acc: 0.5188
Epoch 3/9
----------
synthetic_train Loss: 1.0139 Acc: 0.4977
test Loss: 0.9772 Acc: 0.5281
Epoch 4/9
----------
synthetic_train Loss: 1.0164 Acc: 0.5049
test Loss: 0.9920 Acc: 0.5039
Epoch 5/9
----------
synthetic_train Loss: 1.0005 Acc: 0.5100
test Loss: 0.9546 Acc: 0.5250
Epoch 6/9
----------
synthetic_train Loss: 0.9985 Acc: 0.5150
test Loss: 0.9526 Acc: 0.5289
Epoch 7/9
----------
synthetic_train Loss: 0.9938 Acc: 0.5148
test Loss: 0.9246 Acc: 0.5531
Epoch 8/9
----------
synthetic_train Loss: 0.9942 Acc: 0.5227
test Loss: 0.9191 Acc: 0.5570
Epoch 9/9
----------
synthetic_train Loss: 0.9827 Acc: 0.5248
test Loss: 0.9252 Acc: 0.5414

Training complete in 8m 11s
Best test Loss: 0.919050


/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


...

Epoch 0/9
----------
synthetic_train Loss: 1.1278 Acc: 0.4683
test Loss: 2.4255 Acc: 0.4953
Epoch 1/9
----------
synthetic_train Loss: 1.0836 Acc: 0.4829
test Loss: 1.2408 Acc: 0.3586
Epoch 2/9
----------
synthetic_train Loss: 1.0773 Acc: 0.4746
test Loss: 1.1225 Acc: 0.3578
Epoch 3/9
----------
synthetic_train Loss: 1.0861 Acc: 0.4697
test Loss: 1.0423 Acc: 0.4953
Epoch 4/9
----------
synthetic_train Loss: 1.0724 Acc: 0.4795
test Loss: 1.0434 Acc: 0.4953
Epoch 5/9
----------
synthetic_train Loss: 1.0858 Acc: 0.4683
test Loss: 1.6433 Acc: 0.4953
Epoch 6/9
----------
synthetic_train Loss: 1.0636 Acc: 0.4849
test Loss: 5.2070 Acc: 0.4953
Epoch 7/9
----------
synthetic_train Loss: 1.0595 Acc: 0.4741
test Loss: 2.7045 Acc: 0.4953
Epoch 8/9
----------
synthetic_train Loss: 1.0566 Acc: 0.4810
test Loss: 5.0390 Acc: 0.4953
Epoch 9/9
----------
synthetic_train Loss: 1.0520 Acc: 0.4912
test Loss: 1.4207 Acc: 0.2828

Training complete in 3m 51s
Best test Loss: 1.042328


/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 0/9
----------
synthetic_train Loss: 1.1380 Acc: 0.4448
test Loss: 1.0452 Acc: 0.4953
Epoch 1/9
----------
synthetic_train Loss: 1.0673 Acc: 0.4780
test Loss: 1.1047 Acc: 0.3570
Epoch 2/9
----------
synthetic_train Loss: 1.0918 Acc: 0.4707
test Loss: 1.0889 Acc: 0.4953
Epoch 3/9
----------
synthetic_train Loss: 1.0714 Acc: 0.4609
test Loss: 1.0576 Acc: 0.4953
Epoch 4/9
----------
synthetic_train Loss: 1.0699 Acc: 0.4746
test Loss: 1.0527 Acc: 0.4953
Epoch 5/9
----------
synthetic_train Loss: 1.0647 Acc: 0.4746
test Loss: 1.0519 Acc: 0.4953
Epoch 6/9
----------
synthetic_train Loss: 1.0568 Acc: 0.4814
test Loss: 1.3436 Acc: 0.2633
Epoch 7/9
----------
synthetic_train Loss: 1.0596 Acc: 0.4907
test Loss: 1.0578 Acc: 0.5008
Epoch 8/9
----------
synthetic_train Loss: 1.0495 Acc: 0.4897
test Loss: 1.2633 Acc: 0.4953
Epoch 9/9
----------
synthetic_train Loss: 1.0553 Acc: 0.4790
test Loss: 1.2256 Acc: 0.4953

Training complete in 4m 2s
Best test Loss: 1.045159


/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 0/9
----------
synthetic_train Loss: 1.1345 Acc: 0.4570
test Loss: 1.1030 Acc: 0.4953
Epoch 1/9
----------
synthetic_train Loss: 1.0856 Acc: 0.4624
test Loss: 3.4049 Acc: 0.4953
Epoch 2/9
----------
synthetic_train Loss: 1.0700 Acc: 0.4761
test Loss: 4.1346 Acc: 0.4953
Epoch 3/9
----------
synthetic_train Loss: 1.0642 Acc: 0.4697
test Loss: 1.8396 Acc: 0.3586
Epoch 4/9
----------
synthetic_train Loss: 1.0638 Acc: 0.4595
test Loss: 1.0957 Acc: 0.4953
Epoch 5/9
----------
synthetic_train Loss: 1.0712 Acc: 0.4731
test Loss: 1.0417 Acc: 0.4953
Epoch 6/9
----------
synthetic_train Loss: 1.0714 Acc: 0.4775
test Loss: 1.0850 Acc: 0.4953
Epoch 7/9
----------
synthetic_train Loss: 1.0568 Acc: 0.4780
test Loss: 1.1458 Acc: 0.4953
Epoch 8/9
----------
synthetic_train Loss: 1.0534 Acc: 0.4790
test Loss: 1.3697 Acc: 0.4953
Epoch 9/9
----------
synthetic_train Loss: 1.0536 Acc: 0.5010
test Loss: 1.0752 Acc: 0.4953

Training complete in 3m 52s
Best test Loss: 1.041654
